In [1]:
import pandas as pd
import requests # for making requests to the google maps api
import json # for processing the response from google maps

In [2]:
# to retrieve api_key
with open("credentials","r") as f:
    creds = f.read()

### Schools Data

In [4]:
school_data = pd.read_csv("./data/schools/general-information-of-schools.csv")

In [6]:
# take a look at the data
school_data.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,special_sdp_offered
0,NATIONAL JUNIOR COLLEGE,www.nationaljc.moe.edu.sg,37 HILLCREST ROAD,288913,64661144,na,64684535,na,NJC@MOE.EDU.SG,"BOTANIC GARDENS MRT, TAN KAH KEE MRT, SIXTH AV...",...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,"For the Integrated Programme (Junior High), we..."
1,TEMASEK JUNIOR COLLEGE,http://temasekjc.moe.edu.sg,22 BEDOK SOUTH ROAD,469278,64428066,na,64428762,na,TEMASEK_JC@MOE.EDU.SG,BEDOK MRT,...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,TJC 6-Year Integrated Programme provides a sea...
2,JURONG JUNIOR COLLEGE,http://www.jurongjc.moe.edu.sg,800 CORPORATION ROAD,649809,65624611,na,65624505,na,jjc@moe.edu.sg,"Lakeside, Boon Lay",...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,Chinese,Malay,Tamil,1. Experiential Learning Programmes (ELP) 2. E...
3,ANDERSON JUNIOR COLLEGE,ajc.moe.edu.sg.,4500 ANG MO KIO AVENUE 6,569843,64596822,na,64598734,na,anderson_jc@moe.edu.sg,Yio Chu Kang,...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,na,na,na,"Science research, service-learning, student le..."
4,VICTORIA JUNIOR COLLEGE,http://www.victoriajc.moe.edu.sg/,20 MARINE VISTA,449035,64485011,na,64438337,na,victoria_jc@moe.edu.sg,"Nearest MRT Stations: Kembangan, Eunos",...,FULL DAY,JUNIOR COLLEGE,No,No,No,Yes,Chinese,Malay,Tamil,Arts Programme Beyond Borders Programme Highe...


In [101]:
# add singapore to the end of the address so google maps returns more relevant results
school_data["address"] = school_data.address.apply(lambda x: x+ " SINGAPORE")

In [134]:
school_data.address.head()

0           37 HILLCREST ROAD SINGAPORE
1         22 BEDOK SOUTH ROAD SINGAPORE
2        800 CORPORATION ROAD SINGAPORE
3    4500 ANG MO KIO AVENUE 6 SINGAPORE
4             20 MARINE VISTA SINGAPORE
Name: address, dtype: object

In [3]:
# this function takes an address and uses google maps api to return the latitude and longitude
def geocode(address, creds):
    resp = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address=" + address + "&key=" + creds)
    if len(resp.json()["results"]) == 0:
        return 0,0
    lat = resp.json()["results"][0]["geometry"]["location"]["lat"]
    lng = resp.json()["results"][0]["geometry"]["location"]["lng"]
    return lat,lng

In [105]:
# apply the function to the address column
coords = school_data.address.apply(lambda x: geocode(x,creds))

In [132]:
coords.head()

0    (1.3300068, 103.8044544)
1    (1.3189246, 103.9355622)
2      (1.350886, 103.713267)
3     (1.378043, 103.8457619)
4      (1.306142, 103.919825)
Name: address, dtype: object

In [146]:
# add the coordinates back to the dataframe
school_data["coords"] = coords

In [147]:
# select the useful columns
school_data_final = school_data[["school_name", "address", "coords", "mainlevel_code"]]

In [148]:
school_data_final.head()

,school_name,address,coords,mainlevel_code
0,NATIONAL JUNIOR COLLEGE,37 HILLCREST ROAD SINGAPORE,"(1.3300068, 103.8044544)",MIXED LEVEL
1,TEMASEK JUNIOR COLLEGE,22 BEDOK SOUTH ROAD SINGAPORE,"(1.3189246, 103.9355622)",MIXED LEVEL
2,JURONG JUNIOR COLLEGE,800 CORPORATION ROAD SINGAPORE,"(1.350886, 103.713267)",JUNIOR COLLEGE
3,ANDERSON JUNIOR COLLEGE,4500 ANG MO KIO AVENUE 6 SINGAPORE,"(1.378043, 103.8457619)",JUNIOR COLLEGE
4,VICTORIA JUNIOR COLLEGE,20 MARINE VISTA SINGAPORE,"(1.306142, 103.919825)",JUNIOR COLLEGE


In [149]:
# save the new dataframe to a csv
school_data_final.to_csv("./data/schools/geocoded_school_data.csv",index=False)

### Main Dataset

In [36]:
df = pd.read_csv("./data/hdb_prices/resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv")

In [37]:
df.shape

(65251, 11)

In [38]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


In [39]:
df["full_address"] = df.block.map(str) + " " + df.street_name + " SINGAPORE"

In [40]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,full_address
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4 SINGAPORE
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10 SINGAPORE
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4 SINGAPORE
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10 SINGAPORE
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10 SINGAPORE


In [9]:
coords_list = []
for index, row in df.iterrows():
    try:
        coords = geocode(row.full_address, creds)
    except:
        print(index)
        coords = (0,0)
    coords_list.append([row.full_address,coords])

In [10]:
len(coords_list)

65251

In [16]:
address_df = pd.DataFrame(coords_list, columns=["full_address","coords"])

In [17]:
address_df.to_csv("./data/hdb_prices/address_coordinates.csv", index=False)

In [30]:
df["address_coords"] = address_df.coords

In [41]:
df.shape

(65251, 12)

In [32]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,full_address,address_coords
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4 SINGAPORE,"(1.3752431, 103.8374311)"
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10 SINGAPORE,"(1.373289, 103.8560821)"
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4 SINGAPORE,"(1.3739926, 103.8382463)"
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10 SINGAPORE,"(1.3678117, 103.8551856)"
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10 SINGAPORE,"(1.3716063, 103.8577221)"


In [33]:
df.to_csv("hdb_prices.csv",index=False)

In [47]:
pd.read_csv("./data/hdb_prices/hdb_prices.csv")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,full_address,address_coords
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4 SINGAPORE,"(1.3752431, 103.8374311)"
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10 SINGAPORE,"(1.373289, 103.8560821)"
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4 SINGAPORE,"(1.3739926, 103.8382463)"
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10 SINGAPORE,"(1.3678117, 103.8551856)"
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10 SINGAPORE,"(1.3716063, 103.8577221)"
5,2015-01,ANG MO KIO,3 ROOM,603,ANG MO KIO AVE 5,07 TO 09,67.0,New Generation,1980,64,290000.0,603 ANG MO KIO AVE 5 SINGAPORE,"(1.380163, 103.8357101)"
6,2015-01,ANG MO KIO,3 ROOM,709,ANG MO KIO AVE 8,01 TO 03,68.0,New Generation,1980,64,290000.0,709 ANG MO KIO AVE 8 SINGAPORE,"(1.3711528, 103.8476668)"
7,2015-01,ANG MO KIO,3 ROOM,333,ANG MO KIO AVE 1,01 TO 03,68.0,New Generation,1981,65,293000.0,333 ANG MO KIO AVE 1 SINGAPORE,"(1.3613131, 103.8516526)"
8,2015-01,ANG MO KIO,3 ROOM,109,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,62,300000.0,109 ANG MO KIO AVE 4 SINGAPORE,"(1.3701494, 103.8376575)"
9,2015-01,ANG MO KIO,3 ROOM,564,ANG MO KIO AVE 3,13 TO 15,68.0,New Generation,1985,69,307500.0,564 ANG MO KIO AVE 3 SINGAPORE,"(1.3697651, 103.8593528)"
